# MTH 651: Advanced Numerical Analysis

## Homework Assignment 3

### <span style="color:red;">Write your name here</span>

### Guidelines

* Each student must complete their own assignment individually.
  * Discussing with other students is allowed (encouraged!), but you must write your own answers and code.
  * The use of ChatGTP, Copilot, or other AI assistants is **not allowed**
* The code must run in Colab or JupyterHub without errors.
  * Code that does not run will not receive any credit.
  * I suggest double-checking that your code runs properly in a new session. Sometimes code can be broken but appear to work because of old state in the notebook.

### Google Colab Instructions

* After opening this assignment in Google Colab, click on **"Copy to Drive"**
* Rename the notebook to `student_name_mth_651_assignment_3.ipynb`
    * ⚠️ In the above, replace `student_name` with your name!
* Enter your name above (in the cell below "Homework Assignment")!
* When you are ready to submit your assignment, select "File -> Download -> Download .ipynb" from the Colab menu
* Upload the downloaded `.ipynb` file to Canvas

### Assignment Goals

* The purpose of this assignment is to develop a 2D finite element Poisson solver and analyze its accuracy in the energy and $L^2$ norms.

### Problem Statement

We want to solve the Poisson problem on $\Omega \subseteq \mathbb{R}^2$ with homogeneous Dirichlet boundary conditions
$$
   \begin{aligned}
      -\Delta u &= f && \text{in $\Omega$}  \\
      u &= 0 && \text{on $\partial\Omega$} \\
   \end{aligned}
$$

Recall the finite element formulation of the above problem:

> Find $u_h \in V_h$ such that, for all $v_h \in V_h$
> $$ a(u, v) = (f, v) $$
> where $V_h$ is the space of continuous piecewise linear functions defined on a mesh $\mathcal{T}$ of $\Omega$, and $a(\cdot,\cdot)$ is the bilinear form
> $$ a(u, v) = \int_\Omega \nabla u \cdot \nabla v \, dx $$
> and $(\cdot, \cdot)$ is the $L^2$ inner product,
> $$ (u, v) = \int_\Omega u v \, dx $$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Since we will be working with triangular meshes $\mathcal{T}$, we will need some basic functionality that will help us work with triangular domains.

The first thing we need is a function to approximate integrals over a triangle. We will use a one-point quadrature rule,
$$
    \int_K f(x) \, dx \approx w f(\xi),
$$
where $w$ is the weight, and $\xi$ is the quadrature point.

If the triangle $K$ is defined by vertices $(x_1, x_2, x_3)$, then we take
$$
    \xi := \frac{1}{3}(x_1 + x_2 + x_3),
$$
and
$$
    w := |K| = \text{the area of triangle $K$}.
$$

### Problem 1. (3 points)

Write functions that compute the area of a given triangle, and the one-point quadrature approximation of a function $f$ on a triangle $K$.

The area of the triangle can be computed e.g. using the determinant formula,
$$
    |K| = \frac{1}{2} \det\begin{pmatrix}
        x_1 & y_1 & 1 \\
        x_2 & y_2 & 1 \\
        x_3 & y_3 & 1 \\
    \end{pmatrix}
$$
where $(x_i, y_i)$ are the vertices of $K$. (You can use `np.linalg.det` for this).

In [ ]:
def area(K):
    # K is an array of shape 3x2 containing the vertices of the triangle
    return 0.0

In [ ]:
def quadrature(f, K):
    # f is a function of two variables, called as f(x, y)
    # K is an array of shape 3x2 containing the vertices of the triangle
    return 0.0

### Mesh representation

The mesh $\mathcal{T}$ is represented as a collection of vertices and triangles.
The vertices are stored in an array (of floating point numbers) $V$ of size $N \times 2$.
The triangles are stored in an integer array $T$ of size $n_t \times 3$.

The coordinates of the $i$-th vertex are given by $V[i,:]$ (i.e. by the $i$-th row of $V$).
The **indices** of the $j$-th triangle are given by $T[j,:]$. Note that these are indices into the $V$ array.

The coordinates of the $j$-th triangle are given by $V[T[j,:],:]$, which is an array of size $3 \times 2$.

### Problem 2. (3 points)

Write a function that, given $f$, computes the right-hand side vector $F \in \mathbb{R}^n$ defined by
$$
    F_i := \int_\Omega f \phi_i \, dx,
$$
where $\phi_i$ is the basis function associated with the $i$-th vertex.

Use an **assembly algorithm** to compute these integrals: loop over every triangle $K$, and compute
$$
    \int_K f \phi_i \, dx
$$
for the three basis functions $\phi_i$ that correspond to the vertices of $K$.
Add the contributions to the corresponding entries of the $F$ vector.

The integral can be approximated using one-point quadrature.
Note that $\phi_i(\xi) = \frac{1}{3}$, where $\xi$ is the quadrature point, and $\phi_i$ is any of the three basis functions, and so
$$
    \int_K f \phi_i \, dx \approx \frac{1}{3} \int_K f \, dx
$$



In [ ]:
def make_rhs(f, T, V):
    N = V.shape[0]
    F = np.zeros(N)
    # Assemble F here
    return F

### Problem 3. (3 points)

Write a function that assembles the system matrix $A \in \mathbb{R}^{N \times N}$, where
$$
    A_{ij} := \int_\Omega \nabla \phi_i \cdot \nabla \phi_j \, dx.
$$
Use the assembly algorithm to compute $A$, i.e. compute local stiffness matrices $A_K \in \mathbb{R}^{3 \times 3}$ for each $K \in \mathcal{T}$, and sum the contributions from each triangle.

Note that $A_K$ can be written as
$$
    A_K = |K| G G^T,
$$
where
$$
    G := \begin{pmatrix}
        1 & 1 & 1 \\
        x_1 & x_2 & x_3 \\
        y_1 & y_2 & y_3
    \end{pmatrix}^{-1}
    \begin{pmatrix}
        0 & 0 \\
        1 & 0 \\
        0 & 1
    \end{pmatrix}
$$

This formula can be derived be computing this expression by hand and comparing with the computations from Problem 1.9 from the textbook.

Note that the assembly step can be written as `A[np.ix_(T[it,:], T[it,:])] += A_K`

In [ ]:
def make_stiffness(T, V):
    N = V.shape[0]
    A = np.zeros((N, N))
    # Assemble A here
    return A

### Problem 4. (3 points)

Write a function that eliminates the boundary conditions from the matrix and right-hand side.
Given a list of boundary indices, this function should set the corresponding rows/columns of $A$ to those of the identity matrix, and set the corresponding entries of the right-hand side to zero.

In [ ]:
def eliminate_bcs(A, F, bcs):
    # bcs is a list of boundary indices
    # Modify A and F in-place here
    pass

### Problem 5. (3 points)

Use your functions above to form the linear system $A u = b$ to approximate the solution to the Poisson problem with right-hand side $f \equiv 1$.
Plot the solution using `plt.tricontour(V[:,0], V[:,1], T, u)`.

The mesh should be loaded from the $T$ and $V$ arrays provided here: https://github.com/pazner/mth-651/tree/main/data/meshes

Load them as follows:

```python
T = np.loadtxt("T.txt", dtype=int)
V = np.loadtxt("V.txt")
```

Use the provided function `find_boundary_vertices` to identify the vertices for the boundary conditions.

In [ ]:
def find_boundary_vertices(T):
    edge_count = {}
    for it in range(T.shape[0]):
        for i in range(3):
            e = [T[it,i], T[it,(i+1)%3]]
            e.sort()
            e = tuple(e)
            if e in edge_count:
                edge_count[e] += 1
            else:
                edge_count[e] = 1

    B = []
    for it in range(T.shape[0]):
        for i in range(3):
            e = [T[it,i], T[it,(i+1)%3]]
            e.sort()
            if edge_count[tuple(e)] == 1:
                B.append(T[it,i])
                B.append(T[it,(i+1)%3])

    return B

### Problem 6. (3 points)

Use the function below to generate a mesh of the unit square (and the list of boundary vertices).

Perform a convergence study on a sequence of increasingly refined meshes with $n_x = 4, 8, 16, 32$.
Choose the right-hand side $f$ such that the exact solution $u$ is given by
$$
    u(x,y) = \sin(\pi x) \sin(\pi y).
$$
Use the one-point quadrature to estimate the $L^2$-norm errors.
Compute the rates.
Do the results agree with theory?

In [ ]:
def square_mesh(nx):
    """
    Generates a triangular Cartesian mesh of the unit square with nx vertices in
    each dimension.

    Returns (V, T, B), where V is are the vertex coordinates, T are the triangle
    indices, and B is a list of boundary vertex indices.
    """
    x = np.linspace(0, 1, nx)
    X, Y = np.meshgrid(x, x)
    V = np.stack((X.ravel(), Y.ravel()), axis=1)

    nt = 2*(nx-1)**2
    T = np.zeros((nt, 3), int)

    for iy in range(nx - 1):
        for ix in range(nx - 1):
            v1 = ix + iy*nx
            v2 = ix + 1 + iy*nx
            v3 = ix + (iy + 1)*nx
            v4 = ix + 1 + (iy + 1)*nx
            T[2*ix + iy*2*(nx-1), :] = [v1, v2, v4]
            T[2*ix + 1 + iy*2*(nx-1), :] = [v1, v4, v3]

    B = []
    for i in range(nx):
        B.append(i)
        B.append(i + nx*(nx - 1))
    for i in range(1, nx - 1):
        B.append(nx*i)
        B.append(nx - 1 + nx*i)

    return V, T, B

### Textbook questions. (3 points each)

Problem 4.1

Problem 4.2

Problem 4.8